In [0]:
%sql
--CREATE WIDGET TEXT dateToProcess  DEFAULT 'null'

## ATRIBEC-199 CPU Performance Ranking

Get information about the most recent Smartphone CPU. Generally, High-end smartphones have the latest and most powerful CPU. We'll get the data from the [Nanoreview](https://nanoreview.net/en/soc-list/rating) ranking

#### Step 1 - Import Libraries

In [0]:
pip install BeautifulSoup4

Python interpreter will be restarted.
Requirement already satisfied: BeautifulSoup4 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-a70449c1-019d-45dc-8deb-df107fef7aff/lib/python3.8/site-packages (4.11.1)
Requirement already satisfied: soupsieve>1.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-a70449c1-019d-45dc-8deb-df107fef7aff/lib/python3.8/site-packages (from BeautifulSoup4) (2.3.2.post1)
WARNING: You are using pip version 21.0.1; however, version 22.2.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-a70449c1-019d-45dc-8deb-df107fef7aff/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.

In [0]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

HTTP Request

In [0]:
website = 'https://nanoreview.net/en/soc-list/rating'

In [0]:
response = requests.get(website)

In [0]:
response.status_code

Out[4]: 200

Soup object

In [0]:
soup = BeautifulSoup(response.content, 'html.parser')

In [0]:
results = soup.find_all('tr')

In [0]:
len(results)

Out[7]: 126

In [0]:
# Discard 1st record since it has table labels
results = results[1:]

In [0]:
len(results)

Out[9]: 125

Target Necessary Data and Iterate

* Ranking
* Processor
* Manufacturer
* GPU
* Rating Score

In [0]:
test = results[0]

In [0]:
# Ranking
ranking = [result.find('div', {'class':'tac'}).get_text().strip() for result in results]

# Processor
cpu = [result.find('a', {'style':'font-weight:500;'}).get_text() for result in results]

# Manufacturer
man = [result.find('span', {'class':'text-gray-small'}).get_text() for result in results]

# GPU
gpu = [result.find('div', {'style':'width: 90px; white-space: normal;'}).get_text() for result in results]

# Rating Score
score = [result.find('div', {'class':'table-list-score-box'}).get_text() for result in results]

Create Spark Dataframe

In [0]:
# Convert into Spark DataFrame
spark_df = spark.createDataFrame(pd.DataFrame({'ranking':ranking, 'cpu':cpu, 'manufacturer':man, 'gpu':gpu, 'score':score}))

In [0]:
# Register table so it is accessible via SQL Context
spark_df.createOrReplaceTempView("cpu_ranking") 

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW cpu_ranking_clean AS
SELECT 
  CASE WHEN '$dateToProcess' <> 'null' THEN '$dateToProcess' ELSE CURRENT_DATE END AS process_dt,
  CAST(ranking AS int) AS ranking,
  cpu,
  manufacturer,
  gpu,
  CAST(score AS int) AS score
FROM cpu_ranking

In [0]:
%sql
SELECT * FROM cpu_ranking_clean

process_dt,ranking,cpu,manufacturer,gpu,score
2022-09-13,1,A16 Bionic,Apple,Apple GPU,98
2022-09-13,2,A15 Bionic,Apple,Apple GPU,98
2022-09-13,3,Dimensity 9000 Plus,MediaTek,Mali-G710 MC10,97
2022-09-13,4,Snapdragon 8 Plus Gen 1,Qualcomm,Adreno 730,97
2022-09-13,5,Dimensity 9000,MediaTek,Mali-G710,96
2022-09-13,6,Snapdragon 8 Gen 1,Qualcomm,Adreno 730,94
2022-09-13,7,A14 Bionic,Apple,Apple GPU,92
2022-09-13,8,Snapdragon 888 Plus,Qualcomm,Adreno 660,91
2022-09-13,9,Exynos 2200,Samsung,Samsung Xclipse 920,90
2022-09-13,10,Snapdragon 888,Qualcomm,Adreno 660,90


Save results as a table to use in the model

In [0]:
#dbutils.fs.rm('dbfs:/mnt/jc-analytics-databricks-work/analytics/delta_work/juan_brekes/CPU_ranking/', True)

In [0]:
%sql
--DROP TABLE dv_analytics_work.cpu_ranking

In [0]:
%sql
--CREATE TABLE dv_analytics_work.cpu_ranking
--USING DELTA
--PARTITIONED BY (process_dt)
--LOCATION 'dbfs:/mnt/jc-analytics-databricks-work/dv_analytics_adhoc/ATRIBEC_119_device_clustering_cpu_ranking' 
--AS 
--SELECT * FROM cpu_ranking_clean

In [0]:
%sql
SELECT * FROM dv_analytics_work.cpu_ranking

process_dt,ranking,cpu,manufacturer,gpu,score
2022-08-11,58,Kirin 810,HiSilicon,Mali-G52 MP6,47
2022-08-11,67,Helio G95,MediaTek,Mali-G76 3EEMC4,43
2022-08-11,72,Snapdragon 835,Qualcomm,Adreno 540,42
2022-08-11,78,Exynos 8895,Samsung,Mali-G71 MP20,39
2022-08-11,88,Helio G80,MediaTek,Mali-G52 MP2,33
2022-08-11,93,Tiger T616,Unisoc,Mali-G57 MP1,32
2022-08-11,96,Tiger T700,Unisoc,ARM Mali-G52 MC2,31
2022-08-11,98,Helio G70,MediaTek,Mali-G52 2EEMC2,31
2022-08-11,24,Dimensity 1100,MediaTek,Mali-G77 MC9,71
2022-08-11,63,Dimensity 700,MediaTek,Mali-G57 MC2,44


In [0]:
%sql
DELETE FROM dv_analytics_work.cpu_ranking
WHERE process_dt = CASE WHEN '$dateToProcess' <> 'null' THEN '$dateToProcess' ELSE CURRENT_DATE END

num_affected_rows
0


In [0]:
%sql
INSERT INTO dv_analytics_work.cpu_ranking
SELECT * FROM cpu_ranking_clean

num_affected_rows,num_inserted_rows
125,125


In [0]:
%sql
SELECT 
  *
FROM dv_analytics_work.cpu_ranking

process_dt,ranking,cpu,manufacturer,gpu,score
2022-08-11,58,Kirin 810,HiSilicon,Mali-G52 MP6,47
2022-08-11,67,Helio G95,MediaTek,Mali-G76 3EEMC4,43
2022-08-11,72,Snapdragon 835,Qualcomm,Adreno 540,42
2022-08-11,78,Exynos 8895,Samsung,Mali-G71 MP20,39
2022-08-11,88,Helio G80,MediaTek,Mali-G52 MP2,33
2022-08-11,93,Tiger T616,Unisoc,Mali-G57 MP1,32
2022-08-11,96,Tiger T700,Unisoc,ARM Mali-G52 MC2,31
2022-08-11,98,Helio G70,MediaTek,Mali-G52 2EEMC2,31
2022-08-11,24,Dimensity 1100,MediaTek,Mali-G77 MC9,71
2022-08-11,63,Dimensity 700,MediaTek,Mali-G57 MC2,44


In [0]:
%sql
SELECT 
  process_dt,
  COUNT(*)
FROM dv_analytics_work.cpu_ranking
GROUP BY 1

process_dt,count(1)
2022-08-11,125
2022-09-13,125


In [0]:
dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()

Out[34]: '/Users/juan.brekes@jamcity.com/AVIZ Tickets/00 Device Clustering Project/ATRIBEC-119 Devices Clustering Project/ATRIBEC-119 CPU Performance Ranking'